# Aula 02

Regressão Linear e Logistica

* Visualização dos dados.
* Tratamento em datasets (ETL).
* Correlação.
* Separação para treino, teste e validação.

## Bibliotecas auxiliares - Numpy e pandas

In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Importando dataset

O arquivo do dataset já está no repositório na mesma pasta deste notebook. Caso esteja usando ele separadamente, você pode baixa-lo da fonte citada abaixo.

| Coluna                  | Descrição                                                                           |
| ----------------------- | ----------------------------------------------------------------------------------- |
| **studyName**           | Nome do estudo/projeto de coleta. No caso, “PAL0708” (Palmer LTER, ano 2007–2008).  |
| **Sample Number**       | Número sequencial da amostra. Identificador único dentro do estudo.                 |
| **Species**             | Espécie do pinguim (*Adelie*, *Chinstrap*, *Gentoo*).                               |
| **Region**              | Região geográfica da coleta (sempre “Anvers” = Anvers Island, Antártida).           |
| **Island**              | Ilha específica dentro da região: *Torgersen*, *Biscoe* ou *Dream*.                 |
| **Stage**               | Estágio do ciclo reprodutivo, geralmente “Incubating” (incubação).                  |
| **Individual ID**       | Identificador único para cada pinguim observado.                                    |
| **Clutch Completion**   | Se o casal completou a postura de ovos (“Yes” ou “No”).                             |
| **Date Egg**            | Data da postura do primeiro ovo (formato `yyyy-mm-dd`).                             |
| **Culmen Length (mm)**  | Comprimento do bico (culmen) em milímetros.                                         |
| **Culmen Depth (mm)**   | Profundidade/altura do bico em milímetros.                                          |
| **Flipper Length (mm)** | Comprimento da nadadeira em milímetros.                                             |
| **Body Mass (g)**       | Massa corporal em gramas.                                                           |
| **Sex**                 | Sexo do pinguim (*Male* ou *Female*).                                               |
| **Delta 15 N (o/oo)**   | Valor isotópico δ¹⁵N do tecido do pinguim (indica nível trófico/dieta).             |
| **Delta 13 C (o/oo)**   | Valor isotópico δ¹³C do tecido do pinguim (indica tipo de fonte alimentar/marinha). |
| **Comments**            | Notas adicionais do pesquisador (pouco preenchido).                                 |


Fonte primária:
Gorman, K. B., Williams, T. D., & Fraser, W. R. (2014). Ecological Sexual Dimorphism and Environmental Variability within a Community of Antarctic Penguins (Genus Pygoscelis). PLoS ONE, 9(3), e90081. https://doi.org/10.1371/journal.pone.0090081

In [142]:
df = pd.read_csv('penguins_lter.csv')

### Visualizações Básicas de um dataset

## Tratamento dos dados

## Gráficos e visualizações

### Correlação entre variáveis

---

### Correlação de Pearson  
O **coeficiente de correlação de Pearson (r de Pearson)** mede a intensidade e a direção da relação **linear** entre duas variáveis quantitativas.  
- Varia entre **-1 e +1**:  
  - `+1` indica correlação linear perfeita positiva.  
  - `-1` indica correlação linear perfeita negativa.  
  - `0` indica ausência de correlação linear.  
- Assume que as variáveis são **contínuas, normalmente distribuídas** e com relação **linear**.  
- Sensível a **outliers**, que podem distorcer o valor do coeficiente.  

---

### Correlação de Spearman  
O **coeficiente de correlação de Spearman (ρ, rho)** mede a intensidade e direção da relação **monotônica** (não necessariamente linear) entre duas variáveis.  
- Baseia-se na **ordenação (ranks)** dos dados, não nos valores absolutos.  
- Também varia entre **-1 e +1**, com a mesma interpretação de direção e intensidade.  
- Pode ser usado com variáveis **ordinais** ou quando a relação não é linear.  
- Menos sensível a outliers em comparação ao Pearson.  

---

## Referências  
- Pearson, K. (1895). *Note on regression and inheritance in the case of two parents.* Proceedings of the Royal Society of London, 58, 240–242. https://doi.org/10.1098/rspl.1895.0041 
- Spearman, C. (1904). *The proof and measurement of association between two things.* The American Journal of Psychology, 15(1), 72–101. http://www.jstor.org/stable/1412159

In [155]:
num_df = df.select_dtypes(include=['float64', 'int64'])

In [156]:
from scipy.stats import pearsonr

for col in num_df.columns:
    if col != "Body Mass (g)":
        combined_data = pd.DataFrame({
            'body_mass': num_df['Body Mass (g)'],
            'other_col': num_df[col]
        }).dropna()
        pearson_corr, p_val = pearsonr(combined_data['body_mass'], 
                                        combined_data['other_col'])
        print(f"{col:20} | Pearson r = {pearson_corr:.3f}, p = {p_val:.3e}")

Sample Number        | Pearson r = -0.007, p = 8.968e-01
Culmen Length (mm)   | Pearson r = 0.595, p = 3.808e-34
Culmen Depth (mm)    | Pearson r = -0.472, p = 2.277e-20
Flipper Length (mm)  | Pearson r = 0.871, p = 4.371e-107
Delta 15 N (o/oo)    | Pearson r = -0.538, p = 3.837e-26
Delta 13 C (o/oo)    | Pearson r = -0.375, p = 1.808e-12


In [ ]:
# item_id = 1

# results = px.get_trendline_results(fig)
# print("Trendline coefficients:")
# print(f"a (slope): {results.iloc[item_id]['px_fit_results'].params[1]:.4f}")
# print(f"b (intercept): {results.iloc[item_id]['px_fit_results'].params[0]:.4f}")
# print(f"R-squared: {results.iloc[item_id]['px_fit_results'].rsquared:.4f}")

Trendline coefficients:
a (slope): 34.5734
b (intercept): -3037.1958
R-squared: 0.4116


## Regreção Linear

A **regressão linear** é um dos métodos estatísticos mais utilizados para modelar a relação entre uma variável dependente (resposta) e uma ou mais variáveis independentes (preditoras).  
O objetivo é **ajustar uma linha (ou hiperplano)** que melhor descreva essa relação.

---

## Equação do Modelo

A equação da regressão linear simples é:

$$
y = \beta_0 + \beta_1 x + \varepsilon
$$

Para o caso múltiplo (com várias variáveis preditoras):

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_n x_n + \varepsilon
$$

onde:
- \( y \) é a variável dependente (valor previsto),
- \( \beta_0 \) é o intercepto,
- \( \beta_i \) são os coeficientes de regressão,
- \( x_i \) são as variáveis independentes,
- \( \varepsilon \) é o termo de erro.

---

### Função de Custo (Mínimos Quadrados)

O ajuste dos coeficientes é feito minimizando a soma dos erros quadráticos (MSE – Mean Squared Error):

$$
J(\beta) = \frac{1}{n} \sum_{i=1}^n \left(y_i - \hat{y}_i\right)^2
$$

---

## Visualização

A regressão linear busca encontrar a linha (em 2D) ou o hiperplano (em dimensões maiores) que melhor se ajusta aos dados:

![Regressão Linear](https://upload.wikimedia.org/wikipedia/commons/3/3a/Linear_regression.svg)


In [ ]:
features = [
    'Culmen Length (mm)', 
    'Culmen Depth (mm)', 
    'Flipper Length (mm)', 
    'Delta 15 N (o/oo)', 
    'Delta 13 C (o/oo)'
]

X = df[features].dropna()
y = df.loc[X.index, 'Body Mass (g)']

In [167]:
# print("Intercept:", model.intercept_)
# print("Coefficients:")
# for col, coef in zip(features, model.coef_):
#     print(f"  {col}: {coef:.3f}")

In [168]:
# plt.scatter(y_test, y_pred, alpha=0.7)
# plt.xlabel("Real - Body Mass (g)")
# plt.ylabel("Predicted - Body Mass (g)")
# plt.title("Linear Regression Predictions")
# plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')  # perfect fit line
# plt.show()

### Regressão Logística

## Conceito  
A **regressão logística** é um modelo estatístico amplamente utilizado em **aprendizado de máquina supervisionado** para tarefas de **classificação binária** (por exemplo, prever se um evento ocorre: "sim" ou "não").  
- Diferente da regressão linear, que estima valores contínuos, a regressão logística modela a **probabilidade** de um resultado pertencer a uma determinada classe.  
- Essa probabilidade é obtida pela **função logística (sigmóide)**:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

onde 
$$  
z = \beta_0 + \beta_1 x_1 + ... + \beta_n x_n 
$$  

O resultado da função varia entre **0 e 1**, interpretado como uma probabilidade.

![Função Sigmóide](https://media.licdn.com/dms/image/v2/D5612AQFs75BpVWjxNg/article-cover_image-shrink_600_2000/article-cover_image-shrink_600_2000/0/1680047110274?e=2147483647&v=beta&t=jD-TOg1ENMbSHjshLCk_o-6Ute1a5DlyKU5thKWFAqQ)

---

## Características principais
- Usada para **classificação binária**, mas pode ser estendida para **multiclasse** (via *one-vs-rest* ou *softmax*).  
- Estima os **coeficientes (\(\beta\))** por métodos como **Máxima Verossimilhança**.  
- Pode incluir tanto variáveis **numéricas** quanto **categóricas** (após codificação adequada, ex. one-hot encoding).  
- A saída é uma **probabilidade**, que pode ser convertida em classes usando um **limiar (threshold)**, normalmente 0.5.  

In [ ]:
features = [
    'Culmen Length (mm)',
    'Culmen Depth (mm)',
    'Flipper Length (mm)',
    'Body Mass (g)',   # body mass can help predict sex
    'Species'
]

df_sex = df[features + ['Sex']].dropna()  # drop missing rows

X = pd.get_dummies(df_sex[features], drop_first=True)  # encode categorical
y = df_sex['Sex']

In [169]:
# cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
# cm_df = pd.DataFrame(cm, index=model.classes_, columns=model.classes_)

# # Plot interactive heatmap
# fig = px.imshow(
#     cm_df,
#     text_auto=True,  # show values inside cells
#     color_continuous_scale="Blues",
#     labels=dict(x="Predicted", y="Actual", color="Count"),
#     x=cm_df.columns,
#     y=cm_df.index,
#     title="Confusion Matrix"
# )

# fig.update_layout(yaxis=dict(scaleanchor="x"))  # square cells
# fig.show()